# Script to generate Figure 6 (a)

In [ ]:
import numpy as np

Define path to result files. Define number of simulations for reading in the data

In [ ]:
foldername= "PATH:/to/your/data" # enter directory of log files
num_sims = 20

Define indices of data where to find which information

In [ ]:
event_t = 1 # index of e(t)
policy_t = 2 # index of pi(t)

For one result file determine the point t when $\pi(t) = \pi_p$

In [ ]:
def find_t_look_policy(data):
    for t in range(270):
        if data[t, policy_t] == 1:
            return t
    return 270

We can also determine gaze at the AOI of the reaching target. This can be done for example by determining if the system looks at the patient or at the agent, when it is already transporting the patient, and is, thus, closer than a predefined distance threshold

In [ ]:
def find_t_look(data):
    for t in range(270):
        if data[t, policy_t] == 1 or (data[t, event_t] == 3 and data[t, policy_t] == 0):
            return t
    return 270

In [ ]:
def normalize_t(data, t_look):
    return  (270 - t_look)/100

Read in hand-agent and claw-agent testing data and compute time $t$ of first goal-predictive gaze

In [ ]:
looking_ts_hand = np.zeros((num_sims, 30, 10), dtype='float64')
looking_ts_hand_policy = np.zeros((num_sims, 30, 10), dtype='float64')
for sim in range(num_sims):
    for epoch in range(30):
        for run in range(10):
            filename = foldername + str(sim) + '_epoch' + str(epoch) + '_hand_run' + str(run) + '.txt'
            data = np.loadtxt(filename, dtype='float64', skiprows = 1, delimiter= ', ')
            t_look = find_t_look(data)
            t_look_policy = find_t_look_policy(data)
            looking_ts_hand[sim, epoch, run] = normalize_t(data, t_look)
            looking_ts_hand_policy[sim, epoch, run] = normalize_t(data, t_look_policy)
    print("Processed hand-agent data of simulation " + str(sim))

In [ ]:
looking_ts_claw = np.zeros((num_sims, 30, 10), dtype='float64')
looking_ts_claw_policy = np.zeros((num_sims, 30, 10), dtype='float64')
for sim in range(num_sims):
    for epoch in range(30):
        for run in range(10):
            filename = foldername + str(sim) + '_epoch' + str(epoch) + '_claw_run' + str(run) + '.txt'
            data = np.loadtxt(filename, dtype='float64', skiprows = 1, delimiter= ', ')
            t_look = find_t_look(data)
            t_look_policy = find_t_look_policy(data)
            looking_ts_claw[sim, epoch, run] = normalize_t(data, t_look)
            looking_ts_claw_policy[sim, epoch, run] = normalize_t(data, t_look_policy)
    print("Processed claw-agent data of simulation " + str(sim))

In [ ]:
# Compute mean
looking_ts_hand_mean = np.mean(np.mean(looking_ts_hand_policy, axis=2), axis = 0)
looking_ts_hand_sd = np.std(np.mean(looking_ts_hand_policy, axis=2), axis=0)
looking_ts_claw_mean = np.mean(np.mean(looking_ts_claw_policy, axis=2), axis=0)
looking_ts_claw_sd = np.std(np.mean(looking_ts_claw_policy, axis=2), axis=0)

Plot the gaze-behavior

In [ ]:
# Nice color definitions
colors = [(0.368, 0.507, 0.71), (0.881, 0.611, 0.142),
          (0.56, 0.692, 0.195), (0.923, 0.386, 0.209),
          (0.528, 0.471, 0.701), (0.772, 0.432, 0.102),
          (0.364, 0.619, 0.782), (0.572, 0.586, 0.) ]

In [ ]:
import matplotlib.pyplot as plt

epochs = range(30)

# hand-agent data
x1 = epochs
y1 = looking_ts_hand_mean
err1 = looking_ts_hand_sd
line1= plt.plot(x1, y1, color=colors[0], linewidth=8)
plt.fill_between(x1, y1-err1, y1+err1, alpha=0.5, facecolor=colors[0])

# claw-agent data
x2 = epochs
y2 = looking_ts_claw_mean
err2= looking_ts_claw_sd
line2 = plt.plot(x2, y2, color=colors[3], linewidth=8)
plt.fill_between(x2, y2-err2, y2+err2, alpha=0.5, facecolor=colors[3])

# Event boundaries
plt.plot([0, 30], [0.0, 0.0], 'k')
plt.plot([0, 30], [0.69, 0.69], 'k')
plt.plot([0, 30], [1.7, 1.7], 'k')
plt.plot([0, 30], [2.7, 2.7], 'k')
plt.xlim([0, 30])
plt.ylim([-0.5, 3.0])
plt.yticks([0.35, 1.25, 2.25], ('e_random', 'e_transport', 'e_reach'))
plt.legend((line1[0], line2[0]), ('Hand', 'Claw'))
plt.title('Time t of first activation of \pi_{patient}')
plt.xlabel('# training phases')

# Use this to just visualize the plot
plt.show

# Uncomment this, to store plot in tikz formatz
import tikzplotlib
#tikzplotlib.save("resultPlots/tikz_gaze_tau2.tex")